## 1. 强化学习的特点
- 样本为序列数据，时间上有关联，非独立同分布
- 延迟奖励，没有supervisor给出及时的反馈
- 学习的过程是试错的过程，在探索和利用之间平衡，探索可以更好地估计环境，利用可以更好地获得奖励，但是经常找不到最优解

## 2. 强化学习中的概念
- 轨迹：状态，动作的序列
- 历史：观测，动作，奖励的序列
- 状态vs观测：
    - 当二者等价时，环境为完全可观测，此时可以建模为MDP问题，可用五元组表示，$(S,T,A,R,\gamma)$
    - 当二者不等价时，环境为部分可观测，此时可以建模为POMDP，此时依然具有马尔科夫性质，可以用七元组描述，$(S,T,A,R,\Omega,O,\gamma)$，其中$\Omega(o|s,a)$为观测概率，$O$为观测空间
- episode/trial：一场游戏
- 奖励：标量的反馈信号
- 动作空间：有效动作的集合，离散or连续
- 策略：关于状态的函数$\pi$
    - 随机性策略：$\pi(a|s)=p(a_t=a|s_t=s)$，输入状态，输出动作的概率，具体采取哪个动作需要根据概率采样
    - 确定性策略：$a^*=argmax \pi(a|s)$，输入状态，输出动作（概率最大的）
    - 一般采用随机性策略，多样性
- 价值函数：对未来奖励的预测，用来评估状态的好坏。有以下两种定义
    - $V_{\pi}(s)=E_{\pi}[\sum_{k=0} \gamma^k r_{t+k+1}|s_t=s]$，
    - $Q_{\pi}(s,a)=E_{\pi}[\sum_{k=0} \gamma^k r_{t+k+1}|s_t=s,a_t=a]$
    
- 模型：对环境的的模拟，由状态转移概率和奖励函数组成
    - 状态转移概率为$p_{ss'}^a=p(s_{t+1}=s'|s_t=s,a_t=a)$
    - 奖励函数为$R(s,a)=E[r_{t+1}|s_t=s,a_t=a]$
- 学习：学习的对象是环境
- 规划：环境已知，不需要通过交互即可得到最优解
- 

## 3. agent类型
### 3.1 根据决策方式划分
- value-based agent：显式地学习价值函数，隐式地学习策略，策略从价值函数中推算
- policy-based agent：直接学习策略，输入状态，输出动作（或动作的概率）
- actor-critic agent：结合了价值和策略，同时学习两者，然后通过两者的交互得到最佳的动作

### 3.2 从是否有模型划分
- model-based：不需要与真实世界交互获取数据，而是在模型中学习和规划策略
- model-free：不对环境建模，直接与真实环境交互来学习最优策略，属于数据驱动型方法

## 4. 实验

In [3]:
import gym
from gym import envs
env_specs=envs.registry.all()
envs_ids = [env_spec.id for env_spec in env_specs]
print(envs_ids)


['Copy-v0', 'RepeatCopy-v0', 'ReversedAddition-v0', 'ReversedAddition3-v0', 'DuplicatedInput-v0', 'Reverse-v0', 'CartPole-v0', 'CartPole-v1', 'MountainCar-v0', 'MountainCarContinuous-v0', 'Pendulum-v0', 'Acrobot-v1', 'LunarLander-v2', 'LunarLanderContinuous-v2', 'BipedalWalker-v3', 'BipedalWalkerHardcore-v3', 'CarRacing-v0', 'Blackjack-v0', 'KellyCoinflip-v0', 'KellyCoinflipGeneralized-v0', 'FrozenLake-v0', 'FrozenLake8x8-v0', 'CliffWalking-v0', 'NChain-v0', 'Roulette-v0', 'Taxi-v3', 'GuessingGame-v0', 'HotterColder-v0', 'Reacher-v2', 'Pusher-v2', 'Thrower-v2', 'Striker-v2', 'InvertedPendulum-v2', 'InvertedDoublePendulum-v2', 'HalfCheetah-v2', 'HalfCheetah-v3', 'Hopper-v2', 'Hopper-v3', 'Swimmer-v2', 'Swimmer-v3', 'Walker2d-v2', 'Walker2d-v3', 'Ant-v2', 'Ant-v3', 'Humanoid-v2', 'Humanoid-v3', 'HumanoidStandup-v2', 'FetchSlide-v1', 'FetchPickAndPlace-v1', 'FetchReach-v1', 'FetchPush-v1', 'HandReach-v0', 'HandManipulateBlockRotateZ-v0', 'HandManipulateBlockRotateZTouchSensors-v0', 'HandM

In [4]:
# 在 Gym 库中，一般离散空间用 gym.spaces.Discrete 类表示，连续空间用 gym.spaces.Box 类表示。
env = gym.make('MountainCar-v0')
print('观测空间 = {}'.format(env.observation_space))
print('动作空间 = {}'.format(env.action_space))
print('观测范围 = {} ~ {}'.format(env.observation_space.low,
        env.observation_space.high))
print('动作数 = {}'.format(env.action_space.n))


观测空间 = Box(-1.2000000476837158, 0.6000000238418579, (2,), float32)
动作空间 = Discrete(3)
观测范围 = [-1.2  -0.07] ~ [0.6  0.07]
动作数 = 3


In [5]:
class BespokeAgent:
    def __init__(self, env):
        pass
    
    def decide(self, observation): 
        position, velocity = observation
        lb = min(-0.09 * (position + 0.25) ** 2 + 0.03,
                0.3 * (position + 0.9) ** 4 - 0.008)
        ub = -0.07 * (position + 0.38) ** 2 + 0.07
        if lb < velocity < ub:
            action = 2
        else:
            action = 0
        return action 

    def learn(self, *args): 
        pass
    
agent = BespokeAgent(env)


In [6]:
def play_montecarlo(env, agent, render=False, train=False):
    episode_reward = 0. 
    observation = env.reset() 
    while True: 
        if render: 
            env.render() 
        action = agent.decide(observation)
        next_observation, reward, done, _ = env.step(action) 
        episode_reward += reward 
        if train: 
            agent.learn(observation, action, reward, done) 
        if done: 
            break
        observation = next_observation
    return episode_reward 


In [7]:
env.seed(0) 
episode_reward = play_montecarlo(env, agent, render=True)
print('回合奖励 = {}'.format(episode_reward))
env.close() 
# 小车上山环境有一个参考的回合奖励值 -−110，如果连续 100 个回合的平均回合奖励大于 -−110，则认为这个任务被解决了。

回合奖励 = -105.0
